# Data Preparation: Enriching Threatened Species Index with Historical Weather Data for Time Series Analysis

Name: Zihan

### 中文工作流程总结 (Workflow Summary)

本Jupyter Notebook完整记录了从原始数据加载到最终数据集生成的全过程，主要分为以下四个核心步骤：

1.  **主数据处理 (TSX Index Data Wrangling)**
    * 加载并合并了分布在6个不同CSV文件中的澳大利亚各州及全国的受威胁物种指数（TSX Index）数据。
    * 对数据进行了标准化处理，统一了列名和格式，生成了包含2000年至2021年记录的 `combined_df` 主数据集。

2.  **外部数据策略制定 (External Data Strategy)**
    * 确定通过引入历史天气数据作为外生变量，来丰富和增强后续时间序列模型的预测能力。
    * 选定 Open-Meteo 历史天气API作为数据源，并确定了三个关键天气指标：年平均温度、年总降水量、年总短波辐射量。

3.  **天气数据获取与处理 (Weather Data Acquisition & Processing)**
    * 采用官方 `openmeteo-requests` Python客户端，为其配置了缓存和自动重试功能，以确保数据获取的稳定性和高效性。
    * 以各州首府的经纬度为基准，循环调用API，获取了2000年至2024年的**日度**天气数据。
    * 将海量的日度数据按年份进行聚合（温度求平均，降水与辐射求和），转换为**年度**数据，并计算出全国（National）的平均指标。
    * 对聚合后的数据进行了清理，生成了干净、完整的 `weather_df` 天气数据集。

4.  **最终数据合并与存储 (Final Merge & Storage)**
    * 通过**右合并 (Right Merge)** 的方式，将主数据 (`combined_df`) 与天气数据 (`weather_df`) 合并，生成了包含2000-2024年完整记录的最终数据集 `final_df`。
    * 将 `final_df` 存储为CSV文件 (`Table14_TSX_Table_VIC_version3.csv`)，作为下一阶段**SARIMAX模型预测分析**的直接输入。

### 第 1 步：导入库并设置路径

在这个单元格中，我们导入 `pandas` 和 `os` 库，然后定义包含数据文件的目录路径和文件名列表。这是所有后续操作的准备工作。

In [1]:
# 单元格 1
import os
import pandas as pd

# 定义数据所在的目录
# 使用正斜杠'/'，这在Windows, Mac, 和 Linux上都能很好地工作
data_directory = '01_raw_data/06_tsx_table_vic'

# 定义需要处理的、已经重命名好的文件名列表
target_files = [
    "National.csv",
    "Australian_Capital_Territory.csv",
    "New_South_Wales.csv",
    "South_Australia.csv",
    "Victoria.csv",
    "Western_Australia.csv"
]

print("库已导入，路径已设置。")

库已导入，路径已设置。


### 第 2 步：循环读取、处理并合并数据

这是核心处理步骤。代码会遍历 `target_files` 列表中的每个文件名：
1.  读取对应的 CSV 文件。
2.  根据文件名添加一个 `state` 列。
3.  重命名 `value`, `low`, `high` 列。
4.  按最终要求的顺序重新排列各列。
5.  将处理好的数据（DataFrame）暂存到一个列表中。

In [2]:
# 单元格 2
# 初始化一个空列表，用于存放每个处理好的数据框(DataFrame)
all_dataframes = []

print(f"开始从目录 '{data_directory}' 中处理文件...")

# 循环遍历文件名列表
for filename in target_files:
    filepath = os.path.join(data_directory, filename)

    if not os.path.exists(filepath):
        print(f"警告：文件未找到，已跳过 -> {filepath}")
        continue

    # 1. 读取CSV文件
    df = pd.read_csv(filepath)
    
    # 2. 从文件名中提取州/地区名，并作为新列添加
    state = filename.replace('.csv', '').replace('_', ' ')
    df['state'] = state
    
    # 3. 重命名列以匹配最终的结构
    df.rename(columns={
        'value': 'index_value',
        'low': 'index_conf_low',
        'high': 'index_conf_high'
    }, inplace=True)
    
    # 4. 按照你指定的顺序重新排列列
    df = df[['year', 'state', 'index_value', 'index_conf_low', 'index_conf_high']]
    
    # 5. 将处理好的DataFrame添加到列表中
    all_dataframes.append(df)
    
    print(f"已处理: {filename}")

print("\n所有文件处理完毕。")

开始从目录 '01_raw_data/06_tsx_table_vic' 中处理文件...
已处理: National.csv
已处理: Australian_Capital_Territory.csv
已处理: New_South_Wales.csv
已处理: South_Australia.csv
已处理: Victoria.csv
已处理: Western_Australia.csv

所有文件处理完毕。


### 第 3 步：最终合并与数据类型转换

上一步创建的列表中包含了6个独立的数据集。现在，我们将它们合并成一个单一、完整的数据集，并严格按照您的要求转换每一列的数据类型（例如，年份转为整数，指数值转为浮点数）。

In [3]:
# 单元格 3
# 将列表中的所有DataFrame合并成一个
combined_df = pd.concat(all_dataframes, ignore_index=True)

# 确保最终的数据类型是正确的
combined_df['year'] = combined_df['year'].astype(int)
combined_df['state'] = combined_df['state'].astype(str)
combined_df['index_value'] = combined_df['index_value'].astype(float)
combined_df['index_conf_low'] = combined_df['index_conf_low'].astype(float)
combined_df['index_conf_high'] = combined_df['index_conf_high'].astype(float)

print("数据已成功合并并转换好数据类型。")

数据已成功合并并转换好数据类型。


### 第 4 步：检查最终结果

最后，我们通过两个命令来验证数据是否已按预期准备就绪：
1.  `combined_df.head()`：显示最终数据集的前5行，直观地检查数据内容和格式。
2.  `combined_df.info()`：显示数据集的摘要信息，包括总行数、列名、非空值数量和每列的数据类型，用于确认结构是否正确。

In [4]:
# 单元格 4
# 显示最终DataFrame的前5行，检查数据是否正确
print("--- 最终合并数据 (前5行) ---")
combined_df.head()

--- 最终合并数据 (前5行) ---


,year,state,index_value,index_conf_low,index_conf_high
0,2000,National,1.000000,1.000000,1.000000
1,2001,National,0.894786,0.779023,1.014265
2,2002,National,0.833242,0.717140,0.965727
3,2003,National,0.852811,0.712254,1.026288
4,2004,National,0.799586,0.658812,0.968141


In [5]:
# 单元格 5
# 显示DataFrame的结构信息（列名、非空值数量、数据类型）
print("\n--- 最终数据结构和类型 ---")
combined_df.info()


--- 最终数据结构和类型 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             132 non-null    int32  
 1   state            132 non-null    object 
 2   index_value      132 non-null    float64
 3   index_conf_low   132 non-null    float64
 4   index_conf_high  132 non-null    float64
dtypes: float64(3), int32(1), object(1)
memory usage: 4.8+ KB


### 第 5 步：安装、设置并准备天气数据参数

在这一步，我们将采用更专业、更可靠的方法来获取数据：使用官方的 `openmeteo-requests` Python 客户端。这个单元格将完成所有的数据获取准备工作。

**1. 采用官方客户端的原因：**
-   **智能缓存 (Smart Caching)**：会自动将API的响应结果保存到本地，后续再次运行时速度极快且不消耗API资源。
-   **自动重试 (Automatic Retries)**：如果因网络波动导致请求失败，客户端会自动重新尝试，大大增加了代码的稳定性。

**2. 本单元格执行的准备流程：**
-   **安装库**：确保 `openmeteo-requests` 及其相关依赖已安装。
-   **定义参数**：
    -   **首府坐标**：创建一个包含各州首府及其经纬度的字典。
    -   **API参数**：定义需要获取的天气变量（温度、降水、辐射）和确定的时间范围（2000-01-01 至 2024-12-31）。
-   **设置客户端**：基于上述参数，初始化一个带有缓存和重试功能的 API 客户端，为下一步的数据获取做好准备。

In [6]:
# 1. 安装所需的库 (如果尚未安装)
# 如果在Jupyter中运行，可以在行首加上 ! 符号来执行pip命令
# !pip install openmeteo-requests requests-cache retry-requests numpy pandas

import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# 2. 设置带有缓存和重试功能的 Open-Meteo API 客户端
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# 3. 重新定义我们的参数
# 为每个州/领地定义首府的经纬度
capital_coords = {
    "Victoria": {"lat": -37.81, "lon": 144.96},
    "New South Wales": {"lat": -33.87, "lon": 151.21},
    "South Australia": {"lat": -34.93, "lon": 138.60},
    "Western Australia": {"lat": -31.95, "lon": 115.86},
    "Australian Capital Territory": {"lat": -35.28, "lon": 149.13},
}
start_date = "2000-01-01"
end_date = "2024-12-31"
base_url = "https://archive-api.open-meteo.com/v1/archive"

# API客户端要求变量是一个列表
daily_vars_list = ["precipitation_sum", "temperature_2m_mean", "shortwave_radiation_sum"]

print("✅ 官方API客户端已成功设置！")

✅ 官方API客户端已成功设置！


### 第 6 步：使用官方客户端获取并聚合天气数据

现在，我们将使用上一步设置好的 `openmeteo` 客户端来循环获取每个州 **2000年至2024年** 的全部天气数据。

此代码块会执行以下核心操作：
1.  遍历每个州的首府坐标。
2.  使用官方客户端一次性请求该州25年的全部日度数据。
3.  将返回的数据高效地解析为 Pandas DataFrame。
4.  将日度数据按年份进行**重采样(resample)**和**聚合(aggregate)**：
    -   温度 (`temperature_2m_mean`)：求年**平均值**。
    -   降水 (`precipitation_sum`)：求年**总和**。
    -   辐射 (`shortwave_radiation_sum`)：求年**总和**。
5.  将处理好的年度数据存入一个列表中，为下一步的合并做准备。

In [20]:
# 初始化一个空列表，用于存放每个州处理好的年度天气数据
annual_weather_data_list = []

print("🚀 开始使用官方客户端获取并处理各州天气数据...")

# 循环遍历每一个州
for state, coords in capital_coords.items():
    print(f"\n--- 正在处理 [{state}] ---")
    
    params = {
        "latitude": coords["lat"],
        "longitude": coords["lon"],
        "start_date": start_date,
        "end_date": end_date,
        "daily": daily_vars_list,
        "timezone": "auto"
    }

    try:
        # 使用官方客户端调用API
        responses = openmeteo.weather_api(base_url, params=params)
        response = responses[0]

        # --- 解析返回的数据 ---
        daily = response.Daily()
        
        daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()
        daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
        daily_shortwave_radiation_sum = daily.Variables(2).ValuesAsNumpy()

        # --- 【关键修正】采用官方的 pd.date_range() 方法创建正确的日期索引 ---
        daily_data = {"date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
        )}
        # ----------------------------------------------------------------------

        daily_data["precipitation_sum"] = daily_precipitation_sum
        daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
        daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
        
        daily_df = pd.DataFrame(data=daily_data)
        daily_df.set_index('date', inplace=True)
        
        # --- 年度聚合逻辑 (保持不变) ---
        annual_agg_df = daily_df.resample('YE').agg({
            'temperature_2m_mean': 'mean',
            'precipitation_sum': 'sum',
            'shortwave_radiation_sum': 'sum'
        })
        
        annual_agg_df['state'] = state
        annual_weather_data_list.append(annual_agg_df)
        
        print(f"✅ 已成功获取并处理完 [{state}] 的数据。")

    except Exception as e:
        print(f"❌ 处理 [{state}] 时发生错误: {e}")

print("\n🎉 所有州的数据处理完毕。")

🚀 开始使用官方客户端获取并处理各州天气数据...

--- 正在处理 [Victoria] ---
✅ 已成功获取并处理完 [Victoria] 的数据。

--- 正在处理 [New South Wales] ---
✅ 已成功获取并处理完 [New South Wales] 的数据。

--- 正在处理 [South Australia] ---
✅ 已成功获取并处理完 [South Australia] 的数据。

--- 正在处理 [Western Australia] ---
✅ 已成功获取并处理完 [Western Australia] 的数据。

--- 正在处理 [Australian Capital Territory] ---
✅ 已成功获取并处理完 [Australian Capital Territory] 的数据。

🎉 所有州的数据处理完毕。


### 第 7 步：合并数据并计算全国平均值

现在，我们将上一步列表中所有州的数据合并成一个大的DataFrame。然后，通过对年份进行分组，计算出所有州的平均值，以此作为“National”的数据，并将其追加到最终的DataFrame中。

In [21]:
# 合并所有州的年度天气数据
weather_df = pd.concat(annual_weather_data_list)

# 提取年份作为普通列
weather_df['year'] = weather_df.index.year
weather_df.reset_index(drop=True, inplace=True)

# 计算全国年度平均值
# 我们按年份分组，并计算所有数值列的平均值
national_df = weather_df.groupby('year').mean(numeric_only=True).reset_index()
national_df['state'] = 'National'

# 将全国数据追加到总的DataFrame中
weather_df = pd.concat([weather_df, national_df], ignore_index=True)

# 重命名列以提高可读性
weather_df.rename(columns={
    'temperature_2m_mean': 'annual_mean_temp',
    'precipitation_sum': 'annual_precip_sum',
    'shortwave_radiation_sum': 'annual_radiation_sum'
}, inplace=True)

# 调整列顺序
weather_df = weather_df[['year', 'state', 'annual_mean_temp', 'annual_precip_sum', 'annual_radiation_sum']]

print("已生成最终的天气数据集'weather_df'。")

已生成最终的天气数据集'weather_df'。


### 第 8 步：检查最终的天气数据

最后，我们检查一下 `weather_df` 的内容和结构，确保数据已按预期准备就绪。

In [ ]:
weather_df[weather_df['state'] == "Australian Capital Territory"]

,year,state,annual_mean_temp,annual_precip_sum,annual_radiation_sum


In [13]:
# 显示前几行，检查数据格式
weather_df.head()

,year,state,annual_mean_temp,annual_precip_sum,annual_radiation_sum
0,1999,Victoria,14.716250,0.000000,24.590000
1,2000,Victoria,14.757990,615.799988,5848.040039
2,2001,Victoria,14.526712,564.200012,5662.220215
3,2002,Victoria,14.698013,432.399994,5789.760254
4,2003,Victoria,14.443562,571.000000,5895.100098


In [14]:
# 显示后几行，可以看到我们计算出的'National'数据
weather_df.tail()

,year,state,annual_mean_temp,annual_precip_sum,annual_radiation_sum
125,2020,National,16.554989,899.525024,6237.892578
126,2021,National,16.314037,849.174988,6297.140137
127,2022,National,16.460024,1122.150024,6253.627441
128,2023,National,16.750156,736.325012,6551.685059
129,2024,National,17.195477,731.349976,6567.745117


In [15]:
# 检查数据结构和类型
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  130 non-null    int32  
 1   state                 130 non-null    object 
 2   annual_mean_temp      130 non-null    float32
 3   annual_precip_sum     130 non-null    float32
 4   annual_radiation_sum  130 non-null    float32
dtypes: float32(3), int32(1), object(1)
memory usage: 3.2+ KB


### 第 9 步：数据清理与最终确认

根据之前的分析，我们需要执行最后两项清理工作：
1.  **重新获取缺失数据**：由于API速率限制，`Australian Capital Territory` 的数据未能获取。我们将重新运行第6步和第7步来补全这部分数据。
2.  **剔除异常年份**：数据中出现了一条错误的1999年记录，我们需要将其过滤掉。

完成这些步骤后，`weather_df` 将是一个干净、完整的数据集。

In [22]:
# 清理步骤 1：过滤掉错误的1999年数据
# 我们只保留年份大于等于2000的记录
original_rows = len(weather_df)
weather_df = weather_df[weather_df['year'] >= 2000].copy()

print(f"已过滤掉年份为1999的数据。行数从 {original_rows} 变为 {len(weather_df)}。")

# 清理步骤 2：检查是否所有州的数据都已完整
# 正常情况下应该有6个地区 (5个州/领地 + 1个National)
if len(weather_df['state'].unique()) < 6:
    print("\n检测到有州的数据缺失（可能由于API限制），建议您返回并重新运行第6步和第7步。")
    print("由于缓存机制，重新运行只会请求之前失败的部分，速度会很快。")
else:
    print("\n✅ 所有州/地区的数据均已完整。")

# 最终确认
print("\n--- 清理后的最终数据概览 ---")
weather_df.info()

print("\n--- 数据年份范围 ---")
print(f"从 {weather_df['year'].min()} 年到 {weather_df['year'].max()} 年")

已过滤掉年份为1999的数据。行数从 156 变为 150。

✅ 所有州/地区的数据均已完整。

--- 清理后的最终数据概览 ---
<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 1 to 155
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  150 non-null    int32  
 1   state                 150 non-null    object 
 2   annual_mean_temp      150 non-null    float32
 3   annual_precip_sum     150 non-null    float32
 4   annual_radiation_sum  150 non-null    float32
dtypes: float32(3), int32(1), object(1)
memory usage: 4.7+ KB

--- 数据年份范围 ---
从 2000 年到 2024 年


### 第 10 步：合并主数据与天气数据

现在，我们将 `combined_df` (主数据, 2000-2021) 和 `weather_df` (天气数据, 2000-2024) 合并成一个最终的数据集 `final_df`。

我们将使用**右合并 (Right Merge)**，以天气数据为基准，保留所有年份的记录。这样，在2022-2024年这些只有天气数据的年份里，主数据相关的列（如 `index_value`）将自动填充为 `NaN`。

In [24]:
# 使用 'year' 和 'state' 作为共同的键进行右合并
final_df = pd.merge(combined_df, weather_df, on=['year', 'state'], how='right')

# 检查合并后数据集的头部 (应该显示2000-2001年的完整数据)
print("--- 合并后数据 (头部) ---")
display(final_df.head())

# 检查合并后数据集的尾部 (应该显示2024年数据，且 index_value 为 NaN)
print("\n--- 合并后数据 (尾部) ---")
display(final_df.tail())

--- 合并后数据 (头部) ---


,year,state,index_value,index_conf_low,index_conf_high,annual_mean_temp,annual_precip_sum,annual_radiation_sum
0,2000,Victoria,1.000000,1.000000,1.000000,14.757990,615.799988,5848.040039
1,2001,Victoria,0.851265,0.753910,0.954160,14.526712,564.200012,5662.220215
2,2002,Victoria,0.738937,0.623807,0.873159,14.698013,432.399994,5789.760254
3,2003,Victoria,0.728083,0.578651,0.924713,14.443562,571.000000,5895.100098
4,2004,Victoria,0.647233,0.500643,0.853355,14.277344,621.200012,5806.970215



--- 合并后数据 (尾部) ---


,year,state,index_value,index_conf_low,index_conf_high,annual_mean_temp,annual_precip_sum,annual_radiation_sum
145,2020,National,0.305946,0.202542,0.489085,15.825681,890.140015,6209.967773
146,2021,National,0.331837,0.196334,0.582872,15.465296,883.320007,6234.124023
147,2022,National,NaN,NaN,NaN,15.654364,1109.099976,6152.215820
148,2023,National,NaN,NaN,NaN,16.014750,711.420044,6518.726074
149,2024,National,NaN,NaN,NaN,16.470160,703.160034,6553.233887


### 第 11 步：存储最终合并的数据

我们已经成功地将原始的物种指数数据与通过API获取的天气数据进行了清理、聚合与合并。

最后一步，我们将这个包含了2000-2024年完整记录的 `final_df` DataFrame 保存为一个CSV文件。这个文件将作为我们下一个分析阶段（SARIMAX 预测模型）的输入数据。

代码会自动检查并创建所需的子文件夹 (`01_data_wrangling/02_wrangled_data`)。

In [26]:
import os

# 定义输出文件夹和文件名
output_folder = os.path.join('02_wrangled_data')
output_filename = 'Table14_TSX_Table_VIC_version3.csv'
full_filepath = os.path.join(output_folder, output_filename)

# 确保输出文件夹存在，如果不存在则创建
os.makedirs(output_folder, exist_ok=True)

# 将 final_df 保存为 CSV 文件
# index=False 参数可以防止 pandas 将 DataFrame 的索引写入文件，保持数据整洁
final_df.to_csv(full_filepath, index=False)

print(f"🎉 数据已成功保存至: {full_filepath}")

🎉 数据已成功保存至: 02_wrangled_data\Table14_TSX_Table_VIC_version3.csv
